In [1]:
# the code is apply SCDE+ to minist dataset
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
from tensorflow.keras.models import Sequential

import numpy as np
from sklearn.manifold import SpectralEmbedding
from sklearn.cluster import SpectralClustering

import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
# import datasets mnits from tensorflow.keras
import tensorflow.keras.datasets.mnist as mnist
(x_train, lable), (_, _) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:]))) # make 28*28 img to 784 demission matrix

In [9]:
# make autoencoder
encoding_dim = 32
input_img = keras.Input(shape=(784,))
encoded = layers.Dense(encoding_dim, activation="relu")(input_img)
decoded = layers.Dense(784, activation="sigmoid")(encoded)

# creater autoencoder
autoencoder = keras.Model(input_img, decoded)

# create encoder
encoder = keras.Model(input_img, encoded)

# creater decoder
encoded_input = keras.Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[-1]
decoder = keras.Model(encoded_input, decoder_layer(encoded_input))

In [11]:
# complie and train autoencoder
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.fit(x_train, x_train, epochs=50, batch_size=256,shuffle=True)

Epoch 1/50
235/235 [==============================] - 1s 4ms/step - loss: 0.2762
Epoch 2/50
235/235 [==============================] - 1s 5ms/step - loss: 0.1712
Epoch 3/50
235/235 [==============================] - 1s 4ms/step - loss: 0.1437
Epoch 4/50
235/235 [==============================] - 1s 4ms/step - loss: 0.1283
Epoch 5/50
235/235 [==============================] - 1s 4ms/step - loss: 0.1187
Epoch 6/50
235/235 [==============================] - 1s 4ms/step - loss: 0.1119
Epoch 7/50
235/235 [==============================] - 1s 5ms/step - loss: 0.1068
Epoch 8/50
235/235 [==============================] - 1s 4ms/step - loss: 0.1029
Epoch 9/50
235/235 [==============================] - 1s 4ms/step - loss: 0.1000
Epoch 10/50
235/235 [==============================] - 1s 4ms/step - loss: 0.0979
Epoch 11/50
235/235 [==============================] - 1s 4ms/step - loss: 0.0965
Epoch 12/50
235/235 [==============================] - 1s 4ms/step - loss: 0.0956
Epoch 13/50
235/235 [====

In [12]:
# use encoder to encode raw data
encoded_imgs = encoder.predict(x_train)
# print(encoded_imgs.shape)
# the result is (60000, 32)

In [20]:
# create SpectralEmbedding and get L matrix
# this step take a lot of time:
# TODO: now if I use all 60000 imgs to fit_transform, will make out of memery error; if I use 10000 to transform, it takes more than 10mins
embedding = SpectralEmbedding(n_components=5)
embeded_img = embedding.fit_transform(encoded_imgs[:10000])

In [34]:
# use sc to get clusters
sc = SpectralClustering(n_clusters=10, eigen_solver='arpack', affinity="nearest_neighbors")
sc_result = sc.fit_predict(embeded_img[:10000])

In [36]:
# in first 100 points, all imgs in cluster 1
for i in range(100):
    if sc_result[i] == 1:
        print(lable[i])
# Follow is one test, we can see most 0 is in cluster 1
# 0
# 3
# 0
# 2
# 3
# 2
# 0
# 0
# 3
# 0
# 0
# 0
# 0
# 0
# 0
# 2
# 0
# 0

0
3
0
2
3
2
0
0
3
0
0
0
0
0
0
2
0
0
